In [1]:
#-*- coding:utf-8 -*-'
import numpy as np
import pandas as pd
from TreeNode import *

In [2]:
df = pd.DataFrame(pd.read_csv('DecisionTree.csv'))
map_Age = {'young':0,'middle-aged':1,'old':2}
map_Work = map_House = map_Label = {'no':0,'yes':1}
map_Loan = {'medium':0,'good':1,'excellent':2}
df['Age'] = df['Age'].map(map_Age)
df['Work'] = df['Work'].map(map_Work)
df['House'] = df['House'].map(map_House)
df['Loan'] = df['Loan'].map(map_Loan)
df['Label'] = df['Label'].map(map_Label)

data = df.iloc[:,1:-1].as_matrix()
label = df.iloc[:,-1].as_matrix()
label_cnt = len(map_Label)
feature_cnts = [len(map_Age), len(map_Work), len(map_House), len(map_Loan)]

In [3]:
def entropy(label):
    sum = 0
    for i in range(label_cnt):
        if label.shape[0] == 0:
            return 0
        prop = len(label[label == i])/label.shape[0]
        if prop == 0:
            continue
        sum += prop*np.log2(prop)
    return -sum

In [4]:
def infomation_gain(data, label):
    H_D = entropy(label)
    H_D_A = np.array([0.0 for _ in range(len(feature_cnts))])
    H_A_D = np.array([0.0 for _ in range(len(feature_cnts))])
    for i in range(len(feature_cnts)):
        for j in range(feature_cnts[i]):
            H_D_ij = entropy(label[data[:,i] == j])
            H_D_A[i] += len(label[data[:,i] == j])*H_D_ij/label.shape[0]
            p = len(label[data[:,i] == j])/label.shape[0]
            if p == 0:
                continue
            H_A_D[i] += p*np.log2(p)
    return H_D-H_D_A, -H_A_D

In [5]:
def get_max_pre_prop(label):
    prop = []
    for i in range(label_cnt):
        prop.append(len(label[label_cnt == i]))
    return np.argmax(np.array(prop))

In [6]:
ID3_root = DecisionTreeNode()
ID3_root.train_data = data
ID3_root.label = label
ID3_root.data.append(get_max_pre_prop(label)) 
l = [ID3_root]

while len(l) > 0:
    current_node = l[0]
    l.remove(current_node)
#     print(current_node.train_data.shape)
#     print(current_node.label.shape)
#     print(current_node.train_data)
#     print(current_node.label)
#     print('===========')
    if np.sum(current_node.label-current_node.label[0]) == 0:
        # 表示可以作为叶节点
        continue
    g, _ = infomation_gain(current_node.train_data, current_node.label)
    idx = np.argmax(g)
    current_node.data.append(idx)
    for i in range(feature_cnts[idx]):
        node = DecisionTreeNode()
        node.train_data = current_node.train_data[current_node.train_data[:,idx] == i]
        node.label = current_node.label[current_node.train_data[:,idx] == i]
        node.data.append(get_max_pre_prop(current_node.label[current_node.train_data[:,idx] == i]))
        current_node.addChild(node)
        l.append(node)

In [7]:
ID3_root.visualize()

选择特征：2
选择特征：1
